In [66]:
import sqlite3
import pandas as pd
from lets_plot import *
LetsPlot.setup_html()
from lets_plot.mapping import as_discrete

In [67]:
# File paths for CSV files
driver_loc = r"C:\Users\EVANS\OneDrive\Desktop\PY\ME204\Code\Project\F1_Project\Tables\Driver_Info.csv"
master_loc = r'C:\Users\EVANS\OneDrive\Desktop\PY\ME204\Code\Project\F1_Project\Tables\F1_master_file.csv'
team_loc = r'C:\Users\EVANS\OneDrive\Desktop\PY\ME204\Code\Project\F1_Project\Tables\Team_Info.csv'

# Connect to SQLite database (creates a file if it doesn't exist)
conn = sqlite3.connect('project_f1.db')
cursor = conn.cursor()

# Create driver_info table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS driver_info (
        Driver_id INTEGER PRIMARY KEY,
        Drivers TEXT NOT NULL
    );
""")

# Create team_info table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS team_info (
        Team_id INTEGER PRIMARY KEY,
        Teams TEXT NOT NULL
    );
""")

# Create master_f1 table (Fixed column names)
cursor.execute("""
    CREATE TABLE IF NOT EXISTS master_f1 (
        Drivers_id INTEGER,
        Team_id INTEGER,
        Year INTEGER,
        Australia INTEGER,
        Malaysia INTEGER,
        China INTEGER,
        Turkey INTEGER,
        Bahrain INTEGER,
        Spain INTEGER,
        Monaco INTEGER,
        Canada INTEGER,
        France INTEGER,
        Azerbaijan INTEGER,
        Austria INTEGER,
        Britain INTEGER,
        Germany INTEGER,
        Hungary INTEGER,
        Belgium INTEGER,
        Italy INTEGER,
        Netherlands INTEGER,
        Singapore INTEGER,
        Japan INTEGER,
        Russia INTEGER,
        Emilia_Romagna INTEGER,
        South_Korea INTEGER,
        India INTEGER,
        Abu_Dhabi INTEGER,
        Qatar INTEGER,
        Saudi_Arabia INTEGER,
        USA INTEGER,
        Brazil INTEGER,
        Las_Vagas INTEGER,  
        Miami INTEGER,
        Mexico INTEGER,
        Total_Points INTEGER,
        FOREIGN KEY (Drivers_id) REFERENCES driver_info(Driver_id),
        FOREIGN KEY (Team_id) REFERENCES team_info(Team_id)
    );
""")

conn.commit()
print("Tables created successfully!")

# Load CSV data into DataFrames
driver_df = pd.read_csv(driver_loc)
master_df = pd.read_csv(master_loc)
team_df = pd.read_csv(team_loc)

# Load data into SQLite tables
driver_df.to_sql("driver_info", conn, if_exists="replace", index=False)
team_df.to_sql("team_info", conn, if_exists="replace", index=False)
master_df.to_sql("master_f1", conn, if_exists="replace", index=False)

print("Data loaded successfully!")

# Close the connection
conn.close()

Tables created successfully!
Data loaded successfully!


In [68]:
# Connect to the SQLite database
conn = sqlite3.connect('project_f1.db')

# Query 1: Fetch first 10 rows from driver_info
query = "SELECT * FROM driver_info LIMIT 10;"
df = pd.read_sql(query, conn)
print(df)


         Drivers Driver_id   2011   2012   2013   2014   2015   2016   2017  \
0       A. ALBON     F1_01   None   None   None   None   None   None   None   
1  A. GIOVINAZZI     F1_02   None   None   None   None   None   None  TM_07   
2       A. ROSSI     F1_03   None   None   None   None  TM_13   None   None   
3       A. SUTIL     F1_04  TM_06   None  TM_06  TM_07   None   None   None   
4     B. HARTLEY     F1_05   None   None   None   None   None   None  TM_08   
5       B. SENNA     F1_06  TM_05  TM_09   None   None   None   None   None   
6     C. LECLERC     F1_07   None   None   None   None   None   None   None   
7         C. PIC     F1_08   None  TM_13  TM_19   None   None   None   None   
8       C. SAINZ     F1_09   None   None   None   None  TM_08  TM_08  TM_08   
9       D. KVYAT     F1_10   None   None   None  TM_08  TM_01  TM_08  TM_08   

    2018   2019   2022   2023   2024  
0   None  TM_01  TM_09  TM_09  TM_09  
1   None  TM_18   None   None   None  
2   None   No

In [69]:
# Query 2: Fetch top 2 drivers by Total_Points for each year
query = """
    SELECT Drivers, Teams, Year, Total_Points
    FROM (
        SELECT 
            D.Drivers, 
            T.Teams, 
            M.Year, 
            M.Total_Points,
            RANK() OVER (PARTITION BY M.Year ORDER BY M.Total_Points DESC) AS ranking
        FROM master_f1 M
        JOIN driver_info D ON M.Drivers_id = D.Driver_id 
        JOIN team_info T ON M.Team_id = T.Team_id
    ) ranked_data
    WHERE ranking <= 2
    ORDER BY Year, ranking;
"""
df = pd.read_sql(query, conn)
print(df)
top2_df = df  # Save the result to a DataFrame

          Drivers            Teams  Year  Total_Points
0       S. VETTEL  Red Bull Racing  2011           392
1       J. BUTTON          McLaren  2011           270
2       S. VETTEL  Red Bull Racing  2012           281
3       F. ALONSO          Ferrari  2012           278
4       S. VETTEL  Red Bull Racing  2013           397
5       F. ALONSO          Ferrari  2013           242
6     L. HAMILTON         Mercedes  2014           384
7      N. ROSBERG         Mercedes  2014           317
8     L. HAMILTON         Mercedes  2015           381
9      N. ROSBERG         Mercedes  2015           322
10     N. ROSBERG         Mercedes  2016           385
11    L. HAMILTON         Mercedes  2016           380
12    L. HAMILTON         Mercedes  2017           363
13      S. VETTEL          Ferrari  2017           317
14    L. HAMILTON         Mercedes  2018           408
15      S. VETTEL          Ferrari  2018           320
16    L. HAMILTON         Mercedes  2019           413
17      V.

In [70]:
# Query 3: Fetch all race results with driver and team information
query = """
    SELECT D.Drivers, T.Teams, M.Year, M.Australia, M.Malaysia, M.China, M.Turkey, M.Bahrain, M.Spain, M.Monaco, M.Canada, M.France, 
    M.Azerbaijan, M.Austria, M.Britain, M.Germany, M.Hungary, M.Belgium, M.Netherlands, M.Italy,  
    M.Singapore, M.Japan, M.Russia, M.Emilia_Romagna, M.South_Korea, M.India, M.Abu_Dhabi, M.Qatar, 
    M.Saudi_Arabia, M.USA, M.Brazil, M.Las_Vagas, M.Miami, M.Mexico, M.Total_Points
    FROM master_f1 M
    JOIN driver_info D ON M.Drivers_id = D.Driver_id 
    JOIN team_info T ON M.Team_id = T.Team_id
    ORDER BY M.Year, M.Total_Points DESC;
"""
df = pd.read_sql(query, conn)
print(df)
f1_df = df  # Save the result to a DataFrame
# Close the connection
conn.close()

         Drivers            Teams  Year  Australia  Malaysia  China  Turkey  \
0      S. VETTEL  Red Bull Racing  2011       25.0      25.0   18.0    25.0   
1      J. BUTTON          McLaren  2011        8.0      18.0   12.0     8.0   
2      M. WEBBER  Red Bull Racing  2011       10.0      12.0   15.0    18.0   
3      F. ALONSO          Ferrari  2011       12.0       8.0    6.0    15.0   
4    L. HAMILTON          McLaren  2011       18.0       4.0   25.0    12.0   
..           ...              ...   ...        ...       ...    ...     ...   
270      G. ZHOU           Sauber  2024        0.0       NaN    0.0     NaN   
271    L. LAWSON     Racing Bulls  2024        NaN       NaN    NaN     NaN   
272    V. BOTTAS           Sauber  2024        0.0       NaN    0.0     NaN   
273  L. SARGEANT         Williams  2024        0.0       NaN    0.0     NaN   
274    J. DOOHAN           Alpine  2024        NaN       NaN    NaN     NaN   

     Bahrain  Spain  Monaco  ...  India  Abu_Dhabi 

In [71]:
# Filter rows where Year is 2024
f24_df = f1_df.loc[f1_df["Year"] == 2024]

In [111]:
f24_df["Races_Participated"] = f24_df.iloc[:, 3:-1].notna().sum(axis=1)
f24_df.head()

C:\Users\EVANS\AppData\Local\Temp\ipykernel_23392\3614287565.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f24_df["Races_Participated"] = f24_df.iloc[:, 3:-1].notna().sum(axis=1)


,Drivers,Teams,Year,Australia,Malaysia,China,Turkey,Bahrain,Spain,Monaco,...,Abu_Dhabi,Qatar,Saudi_Arabia,USA,Brazil,Las_Vagas,Miami,Mexico,Total_Points,Races_Participated
251,M. VERSTAPPEN,Red Bull Racing,2024,0.0,NaN,33.0,NaN,26.0,25.0,8.0,...,8.0,26.0,25.0,23.0,31.0,10.0,26.0,8.0,437,25
252,L. NORRIS,McLaren,2024,15.0,NaN,21.0,NaN,8.0,19.0,12.0,...,25.0,9.0,4.0,18.0,16.0,9.0,25.0,18.0,374,25
253,C. LECLERC,Ferrari,2024,19.0,NaN,17.0,NaN,12.0,10.0,25.0,...,15.0,22.0,16.0,30.0,16.0,12.0,22.0,16.0,356,25
254,O. PIASTRI,McLaren,2024,12.0,NaN,6.0,NaN,4.0,6.0,18.0,...,1.0,23.0,12.0,10.0,11.0,6.0,3.0,4.0,292,25
255,C. SAINZ,Ferrari,2024,25.0,NaN,14.0,NaN,15.0,8.0,15.0,...,18.0,13.0,0.0,25.0,4.0,15.0,14.0,25.0,290,25


In [112]:

# Calculate Races_Participated if not already in the DataFrame
f24_df["Races_Participated"] = f24_df.iloc[:, 3:-2].notna().sum(axis=1)

# Set dark theme colors
dark_background = "#000000"  # Black background
text_color = "#FFFFFF"  # White text for contrast
grid_color = "#444444"  # Subtle gridlines

# Create the plot
p = (
    ggplot(f24_df, aes(x="Total_Points", y="Drivers", fill="Races_Participated"))
    + geom_bar(stat="identity", tooltip=layer_tooltips()
               .line("Driver|@Drivers")
               .line("Team|@Teams")
               .line("Total Points|@Total_Points")
               .line("Races Participated|@Races_Participated"))
    + ggtitle("F1 2024 Season - Driver Points")
    + xlab("Total Points")
    + ylab("Drivers")
    + ggsize(1200, 800)
    + theme(
        plot_background=element_rect(fill=dark_background, color=dark_background),
        panel_background=element_rect(fill=dark_background, color=dark_background),
        plot_title=element_text(hjust=0.5, size=16, face="bold", color=text_color),
        axis_text_x=element_text(angle=90, vjust=0.5, color=text_color),
        axis_text_y=element_text(color=text_color),
        axis_title_x=element_text(color=text_color),
        axis_title_y=element_text(color=text_color),
        legend_title=element_text(color=text_color),
        legend_position="right",
        legend_text=element_text(size=9, color=text_color),
        legend_background=element_rect(fill=dark_background, color=dark_background),
        panel_grid_major=element_line(color=grid_color),  # Important gridlines
        panel_grid_minor=element_blank()  # Remove minor gridlines
    )
    + labs(caption="Data Source: 2024 F1 Season")
)

# Save the plot as an interactive HTML file
# ggsave(p, "F1_2024_Driver_Points.html")
p.show()

C:\Users\EVANS\AppData\Local\Temp\ipykernel_23392\3433750552.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f24_df["Races_Participated"] = f24_df.iloc[:, 3:-2].notna().sum(axis=1)


2024 Top 10 Drivers Performance

In [74]:
# Filter top 10 drivers
new_df = f24_df.head(10)
# Arranging the columns in order to the races
new_df= new_df[['Drivers','Teams','Bahrain','Saudi_Arabia','Australia','Japan','China','Miami','Emilia_Romagna','Monaco',
                 'Canada','Spain','Austria','Britain','Hungary','Belgium', 'Netherlands','Italy','Azerbaijan',
                 'Singapore','USA','Mexico','Brazil', 'Las_Vagas','Qatar','Abu_Dhabi']]

# Melt DataFrame for plotting
df_melted = new_df.melt(id_vars=["Drivers", "Teams"], var_name="Race", value_name="Points")

# Sort races in order
df_melted["Race"] = pd.Categorical(df_melted["Race"], 
                                   categories=['Bahrain','Saudi_Arabia','Australia','Japan','China','Miami','Emilia_Romagna','Monaco',
                 'Canada','Spain','Austria','Britain','Hungary','Belgium', 'Netherlands','Italy','Azerbaijan',
                 'Singapore','USA','Mexico','Brazil', 'Las_Vagas','Qatar','Abu_Dhabi'],ordered=True)

# Compute cumulative sum for each driver
df_melted["Cumulative_Points"] = df_melted.groupby("Drivers")["Points"].cumsum()

# Create tooltip text
df_melted["Tooltip"] = df_melted.apply(lambda row: f"Driver: {['Drivers']}\nTeam:{['Teams']} Race: {['Race']}\nRace Points: {['Points']}\nTotal Points: {['Cumulative_Points']}", axis=1)

# Set dark theme colors
dark_background = "#000000"  # Black background
text_color = "#FFFFFF"  # White text for contrast
grid_color = "#444444"  # Subtle gridlines

# Darker color palette
dark_colors = ["#4E79A7", "#F28E2B", "#E15759", "#76B7B2", "#59A14F", 
               "#EDC949", "#AF7AA1", "#FF9DA7", "#9C755F", "#BAB0AC"]
# Create plot
p=(ggplot(df_melted, aes(x="Race", y="Cumulative_Points", color="Drivers", group="Drivers"))
     + geom_line(size=1.2)
     + geom_point(size=3, tooltips=layer_tooltips()
                  .line("Driver|@Drivers")
                  .line("Team|@Teams")
                  .line("Race|@Race")
                  .line("Race Points|@Points")
                  .line("Total Points|@Cumulative_Points"))
     + scale_color_manual(values=dark_colors)
     + ggtitle("Top 10 Drivers of 2024 - F1 Points Progression")
     + ylab("Points")
     + ggsize(1100, 600)  # Increase chart size
     + theme_minimal()
     + theme(
        plot_background=element_rect(fill=dark_background, color=dark_background),
        panel_background=element_rect(fill=dark_background, color=dark_background),
        plot_title=element_text(hjust=0.5, size=16, face="bold", color=text_color),
        axis_text_x=element_text(angle=90, vjust=0.5, color=text_color),
        axis_text_y=element_text(color=text_color),
        axis_title_x=element_text(color=text_color),
        axis_title_y=element_text(color=text_color),
        legend_title=element_text( color=text_color),
        legend_position="right",
        legend_text=element_text(size=9, color=text_color),
        legend_background=element_rect(fill=dark_background, color=dark_background),
        panel_grid_major=element_line(color=grid_color),  # Important gridlines
        panel_grid_minor=element_blank()  # Remove minor gridlines
        
    )
    + labs(caption="Data Source: 2024 F1 Season")
)
# ggsave(p, "Top 10 Drivers of 2024.html")

p.show()

In [75]:
# Calculating mean points for each driver
new_df["Mean_Points"] = new_df.iloc[:, 2:].mean(axis=1)

# Display the result
new_df[["Drivers", "Teams", "Mean_Points"]].reset_index(drop=True)

,Drivers,Teams,Mean_Points
0,M. VERSTAPPEN,Red Bull Racing,18.208333
1,L. NORRIS,McLaren,15.583333
2,C. LECLERC,Ferrari,14.833333
3,O. PIASTRI,McLaren,12.166667
4,C. SAINZ,Ferrari,12.083333
5,G. RUSSELL,Mercedes,10.208333
6,L. HAMILTON,Mercedes,9.291667
7,S. PEREZ,Red Bull Racing,6.333333
8,F. ALONSO,Aston Martin,2.916667
9,P. GASLY,Alpine,1.750000


2024 Teams Performance

In [76]:
new_df = f24_df

# Arranging the columns in order to the races
new_df= new_df[['Drivers','Teams','Bahrain','Saudi_Arabia','Australia','Japan','China','Miami','Emilia_Romagna','Monaco',
                 'Canada','Spain','Austria','Britain','Hungary','Belgium', 'Netherlands','Italy','Azerbaijan',
                 'Singapore','USA','Mexico','Brazil', 'Las_Vagas','Qatar','Abu_Dhabi']]

# Melt DataFrame for plotting
df_melted = new_df.melt(id_vars=["Drivers", "Teams"], var_name="Race", value_name="Points")

# Sort races in order
df_melted["Race"] = pd.Categorical(df_melted["Race"],
                                    categories=['Bahrain', 'Saudi_Arabia', 'Australia', 'Japan', 'China', 'Miami', 'Emilia_Romagna', 'Monaco',
                                                'Canada', 'Spain', 'Austria', 'Britain', 'Hungary', 'Belgium', 'Netherlands', 'Italy', 'Azerbaijan',
                                                'Singapore', 'USA', 'Mexico', 'Brazil', 'Las_Vagas', 'Qatar', 'Abu_Dhabi'],
                                    ordered=True)

# Compute cumulative sum for each team
df_team = (df_melted.groupby(["Teams", "Race"], observed=False)["Points"]
           .sum()
           .groupby(level=0)
           .cumsum()
           .reset_index())

# Merge back to get driver names for tooltips
df_team = df_team.merge(df_melted.groupby(["Teams", "Race"], observed=False)["Drivers"].unique().reset_index(), on=["Teams", "Race"])


# Set dark theme colors
dark_background = "#000000"  # Black background
text_color = "#FFFFFF"  # White text for contrast
grid_color = "#444444"  # Subtle gridlines

# Darker color palette for teams
dark_colors = ["#429bfa", "#0fa306", "#ed1206", "#7f7c70", "#f27b16", "#2ad5cb",
               "#d2a640", "#ebdb00", "#0136fe", "#492cf2", "#492cf2"]

# Create plot with black theme
p=(
    ggplot(df_team, aes(x="Race", y="Points", color="Teams", group="Teams"))
    + geom_line(size=1.5)
    + geom_point(size=3, tooltips=layer_tooltips()
                 .line("Team|@Teams")
                 .line("Race|@Race")
                 .line("Total Points|@Points")
                 .line("Drivers|@Drivers"))
    + scale_color_manual(values=dark_colors)
    + ggtitle("F1 2024 Season - Team Points Progression")
    + ylab("Total Points")
    + ggsize(1000, 600)  # Increase chart size
    + theme(
        plot_background=element_rect(fill=dark_background, color=dark_background),
        panel_background=element_rect(fill=dark_background, color=dark_background),
        plot_title=element_text(hjust=0.5, size=16, face="bold", color=text_color),
        axis_text_x=element_text(angle=90, vjust=0.5, color=text_color),
        axis_text_y=element_text(color=text_color),
        axis_title_x=element_text(color=text_color),
        axis_title_y=element_text(color=text_color),
        legend_title=element_text( color=text_color),
        legend_position="right",
        legend_text=element_text(size=9, color=text_color),
        legend_background=element_rect(fill=dark_background, color=dark_background),
        panel_grid_major=element_line(color=grid_color),  # Important gridlines
        panel_grid_minor=element_blank()  # Remove minor gridlines
        
    )
    + labs(caption="Data Source: 2024 F1 Season")
)
# ggsave(p, "F1 2024 Team Progression.html")

p.show()

In [77]:
top2_df.head()

,Drivers,Teams,Year,Total_Points
0,S. VETTEL,Red Bull Racing,2011,392
1,J. BUTTON,McLaren,2011,270
2,S. VETTEL,Red Bull Racing,2012,281
3,F. ALONSO,Ferrari,2012,278
4,S. VETTEL,Red Bull Racing,2013,397


In [78]:
top2_df
top2_df['Year']=top2_df["Year"].astype(str)
# Determine winner and runner-up for each year
top2_df['Position'] = top2_df.groupby('Year')['Total_Points'].rank(ascending=False, method='min')
top2_df['Status'] = top2_df['Position'].apply(lambda x: 'Winner' if x == 1 else 'Runner-up')

# Darker color palette for teams
bright_colors = ["#4E79A7", "#F28E2B", "#E15759", "#76B7B2", "#59A14F", 
               "#EDC949", "#AF7AA1", "#FF9DA7", "#9C755F", "#BAB0AC"]

# Set dark theme colors
dark_background = "#000000"  # Black background
text_color = "#FFFFFF"  # White text for contrast
grid_color = "#444444"  # Subtle gridlines

# Create the plot
p=(ggplot(top2_df) + 
    geom_bar(aes(x='Year', y='Total_Points', fill='Drivers'), 
             stat='identity',  
             tooltips=layer_tooltips()
                .line('Driver: @Drivers')
                .line('Team: @Teams')
                .line('Status: @Status')
                .line('Total Points: @Total_Points')) +
    labs(title="Runner and Winner of Every Seasons", x="Year", y="Total Points") + 
    scale_fill_manual(values=bright_colors, name="Drivers") + 
    ggsize(1000, 600) +
    theme(
        plot_background=element_rect(fill=dark_background, color=dark_background),
        panel_background=element_rect(fill=dark_background, color=dark_background),
        plot_title=element_text(hjust=0.5, size=16, face="bold", color=text_color),
        axis_text_x=element_text(angle=0, vjust=0.5, color=text_color),
        axis_text_y=element_text(color=text_color),
        axis_title_x=element_text(color=text_color),
        axis_title_y=element_text(color=text_color),
        legend_title=element_text( color=text_color),
        legend_position="right",
        legend_text=element_text(size=9, color=text_color),
        legend_background=element_rect(fill=dark_background, color=dark_background),
        panel_grid_major=element_line(color=grid_color),  # Important gridlines
        panel_grid_minor=element_blank()  # Remove minor gridlines
        
    )
    + labs(caption="Data Source: 2024 F1 Season")
)
# ggsave(p, "Runner and Winner of Every Season.html")

p.show()

In [79]:
df= f1_df

In [80]:
df['Year']=df["Year"].astype(str)
df.drop(columns=['Total_Points'],inplace=True)

In [81]:
# Melt the DataFrame to long format for easier processing
df_melted = df.melt(id_vars=["Drivers", "Teams", "Year"], 
                    var_name="Circuit", 
                    value_name="Points")
# Filter rows where Points > 20
df_filtered = df_melted[df_melted["Points"] > 20]

# Group by Drivers and Circuit, and count occurrences
df_grouped = df_filtered.groupby(["Drivers", "Circuit"]).size().reset_index(name="Count")

# Filter drivers who scored >20 points in the same circuit more than three times
to_5 = df_grouped[df_grouped["Count"] > 3].reset_index(drop=True)

# Display the result
to_5

,Drivers,Circuit,Count
0,L. HAMILTON,Abu_Dhabi,5
1,L. HAMILTON,Brazil,4
2,L. HAMILTON,Britain,5
3,L. HAMILTON,Canada,5
4,L. HAMILTON,China,5
5,L. HAMILTON,Germany,4
6,L. HAMILTON,Hungary,5
7,L. HAMILTON,India,5
8,L. HAMILTON,Italy,4
9,L. HAMILTON,Japan,4
